In [1]:
import keras
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
import pandas as pd
batch_size = 32
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10728693825336462256
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 10573666714
locality {
  bus_id: 1
}
incarnation: 14098024529563471601
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0"
]


# Data Pull
Data comes from ![Kaggle](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data)
My file structure is as follows: 
```
data/
    test/
        unk/
            12500 cat/dog unkown pictures
    train/
        cat/
            11250 cat pictures
        dog/
            11250 dog pictures
    valid/
        cat/
            1250 cat pictures
        dog/
            1250 dog pictures
sample/
    data/
        train/
            cat/
                1138 cat pictures
            dog/
                1138 dog pictures
        valid/
            cat/
                112 cat pictures
            dog/
                112 dog pictures
```

In [2]:
#path = "sample/data/"
path = "data/"

In [40]:
idg = image.ImageDataGenerator(rotation_range=0.1,horizontal_flip=True, channel_shift_range=0.2)
idg_test = image.ImageDataGenerator()
trn_batch = idg.flow_from_directory(path+"train/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
val_batch = idg.flow_from_directory(path+"valid/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
tst_batch1 = idg_test.flow_from_directory(path+"test/", target_size=(224,224), batch_size=batch_size, shuffle=False)
tst_batch2 = idg_test.flow_from_directory(path+"test/", target_size=(224,224), batch_size=batch_size, shuffle=False)
tst_batch3 = idg_test.flow_from_directory(path+"test/", target_size=(224,224), batch_size=batch_size, shuffle=False)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.


In [4]:
#Use MobileNet from Keras
model1 = MobileNet()
#Finetune to look at just two classes instead of 1000
#model.layers.pop()
for layer in model1.layers:
    layer.trainable = False 
m = Dense(1, activation="sigmoid")(model1.layers[-1].output)
model1 = Model(model1.input, m)
#Just guessing on the loss here
model1.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [5]:
#Use ResNet50 from Keras
model2 = ResNet50()
#Finetune to look at just two classes instead of 1000
model2.layers.pop()
for layer in model2.layers:
    layer.trainable = False 
m = Dense(1, activation="sigmoid")(model2.layers[-1].output)
model2 = Model(model2.input, m)
#Just guessing on the loss here
model2.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [6]:
#Use VGG16 from Keras
model3 = VGG16()
#Finetune to look at just two classes instead of 1000
model3.layers.pop()
for layer in model3.layers:
    layer.trainable = False 
m = Dense(1, activation="sigmoid")(model3.layers[-1].output)
model3 = Model(model3.input, m)
#Just guessing on the loss here
model3.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [7]:
model1.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)
model2.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)
model3.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)

Epoch 1/5
704/703 [==============================] - 119s - loss: 0.6372 - acc: 0.8855 - val_loss: 0.5851 - val_acc: 0.9188
Epoch 2/5
704/703 [==============================] - 117s - loss: 0.5462 - acc: 0.9217 - val_loss: 0.5035 - val_acc: 0.9280
Epoch 3/5
704/703 [==============================] - 117s - loss: 0.4754 - acc: 0.9260 - val_loss: 0.4393 - val_acc: 0.9272
Epoch 4/5
704/703 [==============================] - 117s - loss: 0.4195 - acc: 0.9242 - val_loss: 0.3886 - val_acc: 0.9292
Epoch 5/5
704/703 [==============================] - 117s - loss: 0.3738 - acc: 0.9292 - val_loss: 0.3496 - val_acc: 0.9288
Epoch 1/5
704/703 [==============================] - 119s - loss: 0.1158 - acc: 0.9571 - val_loss: 0.0707 - val_acc: 0.9736
Epoch 2/5
704/703 [==============================] - 119s - loss: 0.0779 - acc: 0.9691 - val_loss: 0.0607 - val_acc: 0.9752
Epoch 3/5
704/703 [==============================] - 119s - loss: 0.0707 - acc: 0.9720 - val_loss: 0.0672 - val_acc: 0.9732
Epoch 4/

In [41]:
predictions1 = model1.predict_generator(tst_batch1, steps=(tst_batch1.n/batch_size))
predictions2 = model2.predict_generator(tst_batch2, steps=(tst_batch2.n/batch_size))
predictions3 = model3.predict_generator(tst_batch3, steps=(tst_batch3.n/batch_size))

In [42]:
predictions2

array([[ 0.00269621],
       [ 0.49069789],
       [ 0.02871103],
       ..., 
       [ 0.99999058],
       [ 0.93694067],
       [ 0.83387566]], dtype=float32)

In [43]:
predictions3

array([[ 0.70064133],
       [ 0.99949563],
       [ 0.36763671],
       ..., 
       [ 0.9999944 ],
       [ 0.99345618],
       [ 0.99994302]], dtype=float32)

In [47]:
submission = pd.DataFrame(columns=['id','label'])

In [48]:
#trimming the extremes so nothing is predicting too strongly a 1 or a 0
for i in range(0,len(predictions1)):
    pred_id = tst_batch.filenames[i].split("/")[1].split(".")[0]
    pred_label = (predictions2[i][0]+predictions3[i][0])/2
    #if pred_label > 0.5:
    #    pred_label = 0.95
    #if pred_label < 0.5:
    #    pred_label = 0.05
    #if pred_label == 0.5:
    #    pred_label = 0.5
    if pred_label > 0.97: 
        pred_label = 0.97
    if pred_label < 0.03:
        pred_label = 0.03
    submission = submission.append({"id":pred_id, "label":pred_label}, ignore_index=True)

In [49]:
submission.to_csv("Submission_Dog_Cat_Redux_2Ensemble_17_10_20_06.csv", index=False)

In [21]:
tst_batch.filenames[0]

'unk/8606.jpg'

In [22]:
predictions[0]

array([ 0.52462566], dtype=float32)